ME 639: INTRODUCTION TO ROBOTICS

ASSIGNMENT 5

Navneet Kaur - 18110106

In [8]:
import numpy as np
import math
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from array import *

In [9]:
class PI_controller:

    def __init__(self) -> None:
        self.K = []
        self.error_sum = []
        self.error_buffer = []

    def PI_initialise(self, K):
        self.K = np.array(K)
        self.error_buffer = np.zeros((len(self.K), 2))

    def sumErrorCalc(self,):
        error_sum = []
        for i in range(len(self.error_buffer)):
            error_sum.append(np.sum(self.error_buffer))
        return error_sum

    def controller(self, delt, error):
        self.error_buffer = np.hstack((self.error_buffer[:,1:], np.transpose([error])))
        tau = (self.K[:,0]*error) + (self.K[:,1]*self.sumErrorCalc())
        return tau

Answer 3:

Write a python code incorporating dynamics of the SCARA Manipulator. Employ a simple PI control (you may have to choose proportional and integral gains by trial and error) at each joint independently along with inverse kinematics subroutine (written earlier) to go from one end-effector position to another end-effector position. Both start position and end position can be coded as user inputs (so different points can be specified in different runs by the user). Also, include a simple animation (a crude 2D line-drawing animation is also fine) showing the results from this code.

In [10]:
class SCARA_IK:

    def __init__(self) -> None:
        self.l1 = 1
        self.l2 = 1
        self.l0 = 1
        self.d4 = 0
        self.x3 = 0.5
        self.theta1 = 0
        self.theta2 = 0 
        self.d = [1,1,1]
        pass

    def set_pos(self, d):
        self.d = d
        pass

    def solve_IK(self,):

        l1 = self.l1
        l2 = self.l2
        d = self.d

        def func_IK(theta):
            f =  [
                    l1*math.cos(theta[0]) + l2*math.cos(theta[0]+theta[1]) - d[0],
                    l1*math.sin(theta[0]) + l2*math.sin(theta[0]+theta[1]) - d[1],
            ]
            return f
            # return np.array([f1, f2])
        self.d4 = self.l0 - self.d[2]
        t12 = fsolve(func_IK, [1,2])
        return t12[0], t12[1], self.d4

In [14]:
class SCARA_dynamics(SCARA_IK, PI_controller):
    def __init__(self) -> None:
        super().__init__()
        self.m1 = 1
        self.m2 = 1
        self.m4 = 1
        self.EF = []
        pass

    def set_EFpos(self, theta1, theta2, d4):
        self.EF = [theta1, theta2, d4]
        pass

    def solve(self,):        
        t_max = 100
        t_step = 1000
        delt = t_max/t_step
        Q = np.zeros((t_step,3))
        Qd = np.zeros((t_step,3))
        Qdd = np.zeros((t_step,3))
        
        alpha = (self.m1*self.l1**2/3) + (self.l1**2*self.m1/4) + (self.l1**2*self.m2) + (self.l1**2*self.m4)
        beta = (self.l2**2*self.m2/3) + (self.l2**2*self.m4) + (self.m2*self.l2**2/4)
        gamma = (self.l1*self.l2*self.m4) + (self.l1*self.m2*self.l2/2)
        
        for i in range(1,t_step):
            error = self.EF - Q[i-1]
            tau = self.controller(delt, error)

            M = np.array([
                [alpha + beta + (2*gamma*np.cos(Q[i-1][1])), beta+ (gamma*np.cos(Q[i-1][1])), 0],
                [beta + (gamma*np.cos(Q[i-1][1])), beta, 0],
                [0, 0, self.m4]
            ])
            C = np.array([
                [-gamma*np.sin(Q[i-1][1])*Qd[i-1][1], -gamma*np.sin(Q[i-1][1])*(Qd[i-1][0]+Qd[i-1][1]), 0],
                [gamma*np.sin(Q[i-1][1])*Qd[i-1][0], 0, 0],
                [0,0,0]
            ])
            G = np.array([0,0,self.m4*9.8])

            T = tau - G - np.matmul(C,Qd[i-1].transpose())

            qdd = np.matmul(np.linalg.inv(M),T)


            # qdd = fsolve(qdd_solve, [0,0,0])
            qd = Qd[i-1] +(qdd*delt)
            q = Q[i-1] + (qd*delt) + (delt**2*qdd/2)
            q = q%(2*np.pi)
            Q[i,:] = q[:]
            Qd[i,:] = qd[:]
            Qdd[i,:] = qdd[:]

        return Q, Qd, Qdd

scara_ik = SCARA_IK()
scara_ik.set_pos([1.8,1.5,0.7])
theta1, theta2, d4 = scara_ik.solve_IK()
scara = SCARA_dynamics()
K = [[3,0.5],
    [5,0.2],
    [0.3,0.2]]
scara.PI_initialise(K)
scara.set_EFpos(theta1, theta2, d4)
Q, Qd, Qdd = scara.solve()

C:\Users\DOLL\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Answer 4:

Write a python code incorporating dynamics of the PUMA Manipulator. Employ a simple PI control (you may have to choose proportional and integral gains by trial and error) at each joint independently along with inverse kinematics subroutine (written earlier) to go from one end-effector position to another end-effector position. Both start position and end position can be coded as user inputs (so different points can be specified in different runs by the user). Also, include a simple animation (a crude 2D line-drawing animation is also fine) showing the results from this code

In [17]:
class PUMA(PI_controller):

    def __init__(self) -> None:
        super().__init__()
        self.l1 = 1
        self.l2 = 1
        self.l3 = 1
        self.m1 = 1
        self.m2 = 1
        self.m3 = 1
        self.EF = []
        pass

    def PUMA_IK(self,pos):
        theta1 = np.arctan2(pos[1],pos[0])
        r = np.sqrt(pos[0]**2 + pos[1]**2)
        s = pos[2] - self.l1
        D = (r**2 + s**2 - self.l2**2 - self.l3**2)/(2*self.l2*self.l3)
        theta3 = np.arctan2(np.sqrt(1-D**2),D)
        theta2 = np.arctan2(s,r) - np.arctan2(self.l3*np.sin(theta3), self.l2+ self.l3*np.cos(theta3))
        return theta1, theta2, theta3

    def set_EFpos(self, theta1, theta2, theta3):
        self.EF = [theta1, theta2, theta3]
        pass

    def solve(self,):        
        t_max = 100
        t_step = 1000
        delt = t_max/t_step
        Q = np.zeros((t_step,3))
        Qd = np.zeros((t_step,3))
        Qdd = np.zeros((t_step,3))
        a1 = self.m2*self.l2**2/4 + self.m3*self.l2**2
        a2 = self.m3*self.l3**2/4
        a3 = self.m3*self.l3*self.l2/2
        b1 = (self.m2*self.l2/2 + self.m3*self.l2)*9.8
        b2 = self.m3*self.l3*9.8/2
        for i in range(1,t_step):
            error = self.EF - Q[i-1]
            tau = self.controller(delt, error)

            M = np.array([
                [self.m1*self.l1**2/3 + a1*np.cos(Q[i-1][1])**2 + a2*np.cos(Q[i-1][2])**2 + 2*a3*np.cos(Q[i-1][1])*np.cos(Q[i-1][2]), 0, 0],
                [0, self.m2*self.l2**2/3 + self.m3*self.l3**2/3 + a1, a3*np.cos(Q[i-1][2] - Q[i-1][1]) - self.m3*self.l3**2/3],
                [0, a3*np.cos(Q[i-1][2] - Q[i-1][1]) - self.m3*self.l3**2/3, self.m3*self.l3**2/3 + a2]
            ])
            C = np.array([
                [-a3*Qd[i-1][1]*np.sin(Q[i-1][1])*np.cos(Q[i-1][2]) - a3*Qd[i-1][2]*np.cos(Q[i-1][1])*np.sin(Q[i-1][2]) - 0.5*(a1*Qd[i-1][1]*np.sin(2*Q[i-1][1])) + a2*Qd[i-1][2]*np.sin(2*Q[i-1][2]), -0.5*a1*Qd[i-1][0]*np.sin(2*Q[i-1][1]) - a3*Qd[i-1][0]*np.sin(Q[i-1][1])*np.cos(Q[i-1][2]), -0.5*a2*Qd[i-1][0]*np.sin(2*Q[i-1][2]) - a3*Qd[i-1][0]*np.cos(Q[i-1][1])*np.sin(Q[i-1][2])],
                [0.5*a1*Qd[i-1][0]*np.sin(2*Q[i-1][1]) + a3*Qd[i-1][0]*np.sin(Q[i-1][1])*np.cos(Q[i-1][2]), 0, a3*Qd[i-1][2]*np.sin(Q[i-1][1] - Q[i-1][2])],
                [0.5*a2*Qd[i-1][0]*np.sin(2*Q[i-1][2]) + a3*Qd[i-1][0]*np.cos(Q[i-1][1])*np.sin(Q[i-1][2]), -a3*Qd[i-1][1]*np.sin(Q[i-1][1] - Q[i-1][2]), 0]
            ])
            G = np.array([0, b1*np.cos(Q[i-1][1]), b2*Q[i-1][2]])

            T = tau - G - np.matmul(C,Qd[i-1].transpose())

            qdd = np.matmul(np.linalg.inv(M),T)


            # qdd = fsolve(qdd_solve, [0,0,0])
            qd = Qd[i-1] +(qdd*delt)
            q = Q[i-1] + (qd*delt) + (delt**2*qdd/2)
            q = q%(2*np.pi)
            Q[i,:] = q[:]
            Qd[i,:] = qd[:]
            Qdd[i,:] = qdd[:]

        return Q, Qd, Qdd

puma = PUMA()
theta1, theta2, theta3 = puma.PUMA_IK([1.8,1.5,0.7])
puma.set_EFpos(theta1, theta2, theta3)
K = [[0.2,2],
    [3,0],
    [0,2]]
puma.PI_initialise(K)
Q, Qd, Qdd = puma.solve()

<ipython-input-17-f2f72e6f7575>:19: RuntimeWarning: invalid value encountered in sqrt
  theta3 = np.arctan2(np.sqrt(1-D**2),D)
<ipython-input-17-f2f72e6f7575>:63: RuntimeWarning: invalid value encountered in remainder
  q = q%(2*np.pi)
